<a href="https://colab.research.google.com/github/Austin-jc/dish2shoppinglist/blob/main/crf_nyt_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorials:
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/

https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

In [ ]:
import tensorflow as tf
from tensorflow import keras

import gensim
import numpy as np
import pandas as pd
import re
tf.__version__

'2.3.0'

Word representation.  Need dense representation  for each word.  Also want to extract some sort of meaning from the characters in each word since most words dont even have a pretrained embedding.  This is like if the word starts with a capital or ends with -ly or -ed etc

Word representation is a concatenation of word embeddings and character features.  the nyt open article for the food thing uses hand crafted functions, but the tutorial im following https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html uses a fancy approach of a bi lstm

In [ ]:
pretrained_embed_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/Colab Data/GoogleNews-vectors-negative300.bin', binary = True)
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Colab Data/nyt-ingredients-snapshot-2015.csv",  engine='python', error_bad_lines=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#load the nyt food data set

newData = data[data["input"].notnull()]
#print (newData)
newData = newData.fillna('')
# this is an array of array(sentences)
corpus = [row.split(" ") for row in newData['input'].replace(',', '')]
plain_text=''
for row in newData['input'].replace(',', ''):
  plain_text = plain_text +row

# make vocab
vocab = []
sentence_lengths = []
for s in corpus:
    sentence_lengths.append(len(s))
    for w in s:
      w = w.replace(",", '')
      #print(w)
      if w not in vocab:
        vocab.append(w)

#create vocab mapping
word2idx = {w:idx for (idx,w) in enumerate(vocab)}

In [ ]:
#what is this even doing lol?  If my guess is right it should load the vocab from corpus into the embeddings.  
embed_model = gensim.models.Word2Vec(corpus, min_count=1)

In [ ]:
# make the gensim embeds usable in tf/keras
embed_matrix = [embed_model.wv[embed_model.wv.index2word[v]] 
                for v in range(len(embed_model.wv.vocab))]

embed_matrix = np.array(embed_matrix)
embed_matrix.shape[1]

100

Character level word representation is carried out by bi lstm which i think is just 2 lstm put to gether?? bidirectional_dynamic_rnn in tf?

In [ ]:
#word embeddings
embedding_dim = embed_model.vector_size
wordEmbed = keras.layers.Embedding(embed_matrix.shape[0],
                                        embed_matrix.shape[1],
                                        embeddings_initializer = keras.initializers.Constant(embed_matrix),
                                        trainable = False)
embedding = tf.Variable(initial_value=embed_matrix, trainable=False)
#character embeddings
wordEmbed(162)
#word Context


<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-0.7084012 , -0.9029062 , -1.3594178 , -1.4816473 ,  0.6763792 ,
        0.49468818, -0.44584686,  1.0539653 , -0.42189285,  0.03777173,
        2.4255977 , -0.16810279, -1.5660539 ,  1.3685815 , -0.98313576,
       -0.8159045 , -1.5598844 , -1.4706031 ,  0.5025849 ,  0.2593317 ,
       -0.19089563,  0.9320042 , -1.4718076 , -0.02835625, -0.29834908,
        0.23604564,  0.5737773 , -0.10224696, -0.49665955,  1.3177564 ,
        1.1715152 ,  0.18341163,  0.764759  ,  0.9195285 ,  1.8607581 ,
       -1.5660838 , -0.81668925,  0.5733979 ,  0.62845   , -0.32894623,
       -0.06040537, -0.7872032 ,  1.8584213 ,  1.6279293 ,  1.5294938 ,
        1.0047805 ,  2.5685136 ,  0.60202533, -0.7317871 ,  0.52935505,
       -0.45678493,  1.074107  , -1.7215221 , -0.99702895, -0.59858286,
       -1.1978964 , -2.2086177 ,  1.3718534 , -1.1028216 , -0.59805727,
        0.01331429, -0.2817378 , -1.5047516 ,  0.45324397, -0.6499997 ,
        0.679953

In [ ]:
#character level representation - do we really need character embeds???
#ill use it as a padding?
char2idx = {c:i+1 for i, c in enumerate(sorted(set(plain_text)))}
char2idx[0] = 0
idx2char = np.array(vocab)


In [ ]:
class LstmCrf(keras.Model):

  def __init__(self, embed_matrix, word2idx, char2idx):
    super(LstmCrf, self).__init__()
    self.word_embedding = keras.layers.Embedding(embed_matrix.shape[0],
                                            embed_matrix.shape[1],
                                            mask_zero=True
                                            # embeddings_initializer = keras.initializers.Constant(embed_matrix),
                                            # trainable = False
                                            )
    self.char_embedding = keras.layers.Embedding(input_dim=len(char2idx), output_dim=50)
    self.char_bilstm = keras.layers.Bidirectional(keras.layers.LSTM(50))

    self.word_context_lstm = keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True))

    # compute scores
    self.dense = keras.layers.Dense(5)
    self.softmax = keras.layers.Dense(5, activation = keras.activations.softmax)
    #crf??
    # logits = keras.layers.dense()
    # self.crf = keras.layers.Con

  def call(self, input):
    #ipnut is some strings.  split it into array and get word embeds
    if type(input) == str:
      s = input.split(' ')
    else:
      #pad sentences
      s = keras.preprocessing.sequence.pad_sequences(input)
    
    w_rep = [self.word_embedding(word2idx[w]) for w in s]
    w_rep = np.asarray(w_rep)
    #get character level rep
    encoded_s = []
    for w in s:
      encoded_s.append([char2idx[c] for c in w])
    #pad the words to max sentence length
    padded_s = keras.preprocessing.sequence.pad_sequences(encoded_s)
    print(padded_s)
    c_rep = []
    for w in padded_s:
      c_emb_matrix = [self.char_embedding(float(c)) for c in w] #shape of nchars, embed dim
      #print(np.shape(c_emb_matrix))
      w_vec = self.char_bilstm(np.reshape(c_emb_matrix, (1,np.shape(c_emb_matrix)[0], np.shape(c_emb_matrix)[1])))  # this uses the embeds as features... not sure how it;ll go 
      #print(np.shape(w_vec))
      c_rep.append(w_vec)

    #concatenate word and character representations
    c_rep = np.asarray(c_rep).reshape(w_rep.shape[0], w_rep.shape[1])
    w_embeddings = tf.concat([w_rep, c_rep], axis=-1)
    #print("we shape: " + str(tf.reshape(w_embeddings, (1, w_embeddings.shape[0], w_embeddings.shape[1]))))
    #get contextual info between words in the sentence
    #the input is 2x200 2 words x w2v + chars 
    # w_embeddings = tf.reshape(w_embeddings, ( w_embeddings.shape[0], w_embeddings.shape[1], 1))
    w_embeddings = tf.reshape(w_embeddings, (1, w_embeddings.shape[0], w_embeddings.shape[1]))
    context_w_rep = self.word_context_lstm(w_embeddings)

    scores = self.dense(context_w_rep)
    output = self.softmax(scores)
    return output

m = LstmCrf(embed_matrix,word2idx, char2idx)
m("cocoa powder cake")


[[ 0 65 77 65 77 63]
 [78 77 85 66 67 80]
 [ 0  0 65 63 73 67]]


<tf.Tensor: shape=(1, 3, 5), dtype=float32, numpy=
array([[[0.19989519, 0.20216486, 0.19847456, 0.20292261, 0.19654281],
        [0.19973345, 0.201445  , 0.20009002, 0.20244011, 0.19629139],
        [0.19985119, 0.20183013, 0.20088762, 0.20064627, 0.19678478]]],
      dtype=float32)>

In [ ]:
m.compile(optimizer='adam',
          loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
          metrics=['accuracy'])


In [ ]:
#map qty to fractions
dec2frac = {
    "3/4": '0.75',
    "2/3": '0.67',
    "1/2": "0.5",
    "1/3": '0.33',
    "1/4": '0.25',
    "1/5": '0.2',
    "1/8": '0.13' 
  }
def qtyChecker(s, qty):
  if s.isnumeric() == False and s not in dec2frac:
    return False
  int_val = np.floor(qty)
  dec_val = qty%1
  # print("s: " + s+ "q: "+ str(qty)  + " iv: "+str(int_val)+"dv: "+ str(dec_val))
  if s == str(int(int_val)):
    return True
  elif s not in dec2frac:
    return False
  elif dec2frac[s] == (str(dec_val)):
    return True
  else:
    return False

# for q in qty:
#   if q%1 != 0:

tag2idx = {
    'other' : 0,
    'item' : 1,
    'qty' : 2,
    'unit' : 3,
    'comment' : 4
}
idx2tag = np.array(list(tag2idx.items()))

def parsePhrase(info):
  """
  takes in a row from the pd data frame and outputs the tag for each space seperated item in the input column
  """
  tag = []
  for w in info["input"].split(' '):
    # print(w)
    found_unit = False
    w = re.sub('[(),]','', w)
    if w in info["name"]:
      tag.append(tag2idx['item'])
    elif info["unit"] in w:
      tag.append(tag2idx['unit'])
      found_unit = True
    elif qtyChecker(w, info["qty"]) and found_unit == False:
      tag.append(tag2idx['qty'])
    elif info["comment"] != None:
      if w in info["comment"]:
        tag.append(tag2idx['comment'])
    else:
      tag.append(tag2idx['other'])
  return np.asarray(tag)

y = []
for r in range(len(newData)):
  # print(str(r) + newData.iloc[r]['input'])
  y.append(parsePhrase(newData.iloc[r]))



In [ ]:
y = np.asarray(y)
def splitData(x, labels, train_portion = .75, test_portion = .2, valid_portion = .05):
  """
  x is the input sentence (df['input])
  returns in order: train x,y - test x,y - valid x,y according the the percentage given (default .75/.2/.05)
  """
  train_idx = int(round(x.size*train_portion))
  test_idx =  int(round(x.size*(train_portion+test_portion)))

  train_x = x[:train_idx].values
  train_x = np.asarray([np.asarray(s.split(' ')) for s in train_x])
  test_x = x[train_idx+1:test_idx].values
  test_x = np.asarray([np.asarray(s.split(' ')) for s in test_x])
  valid_x = x[test_idx : x.size].values
  valid_x = np.asarray([np.asarray(s.split(' ')) for s in valid_x])
  
  train_y = labels[:train_idx]
  test_y = np.array(labels[train_idx+1:test_idx])
  valid_y = np.array(labels[test_idx:x.size])
  
  return train_x, train_y, test_x, test_y, valid_x, valid_y


In [ ]:
print(y[0])
print(idx2tag[y[0],0])
print(newData.iloc[0]['input'])
'butternut' in 'butternut squash'

[2 2 3 4 4 4 4 1 1 4 2 4 4 4 1 4]
['qty' 'qty' 'unit' 'comment' 'comment' 'comment' 'comment' 'item' 'item'
 'comment' 'qty' 'comment' 'comment' 'comment' 'item' 'comment']
1 1/4 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted


True

In [ ]:
epochs = 10
lr = 0.1

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()


train_x, train_y, test_x, test_y, valid_x, valid_y = splitData(newData['input'], y)

model= LstmCrf(embed_matrix,word2idx, char2idx)

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for s in train_x:
        with tf.GradientTape() as tape:
            y_pred = model(s)
            # Compute reconstruction loss
            loss = mse_loss_fn(train_y, reconstructed)
            loss += sum(model.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))


Start of epoch 0


ValueError: ignored

In [ ]:
model.compile(optimizer='adam',
          loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
          metrics=['accuracy'])
model.fit(
    x= train_x,
    y = train_y,
    epochs = 20
)

ValueError: ignored

In [ ]:
type(y[0])

numpy.ndarray